In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))
from source.beziercurve import BezierCurve
from source.diffrasterizer import rasterize_spheres
from source.utils import load_scene
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import plotly 
import plotly.graph_objects as go
from PIL import Image
from source.sampler import ImageSampler
from einops import rearrange, repeat

In [2]:
%cd ..

/home/alberts/Documents/EPFL/MA2/Visual_Intelligence/clipasso3d


/home/alberts/miniconda3/envs/3dsketch/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# autoload file changes
%load_ext autoreload
%autoreload 2

In [ ]:
# 1) load scene
image = 9
device = 'cuda'
W, H = 224, 224

# --- Prepare data sampler for a single scene ---
data_name = 'chair'
transforms_json = "data/{}/transforms_train.json".format(data_name)
image_dir = 'data/{}'.format(data_name)
sampler = ImageSampler(
    transforms_json=transforms_json,
    image_dir=image_dir,
    width=W,
    height=H,
    total_iters=0,  # unused in random sampling
    device=device,
)

batch_size = 8
w2c = sampler.w2c_all[:batch_size]
K = sampler.K

#file_paths, c2w_mats, K = load_scene("data/transforms_train.json", W, H, device=device)
#w2c = torch.linalg.inv(c2w_mats[9]) 
#w2c = repeat(w2c, "c1 c2 -> b c1 c2", b=batch_size)  # (B,4,4)

# 3) instantiate Bezier curve around scene center
#    here control points in [-1,1]^3 space around the origin
init_pts1 = torch.tensor([
    [-1.0, 0.0, 0.0],
    [-0.5, 0.5, 0.2],
    [ 0.5,-0.5, 0.3],
    [ 1.0, 0.0, 0.0]
], dtype=torch.float16, device='cuda')

init_pts2 = torch.tensor([
    [1.0, 0.0, 0.0],
    [0.5, 0.5, 0.2],
    [ 0.5,-0.5, 0.3],
    [ 1.0, 0.0, 0.0]
], dtype=torch.float16, device='cuda')

bez1 = BezierCurve(init_pts1, thickness=0.05, overlap=0.9, arc_samples=300, device=device)
bez2 = BezierCurve(init_pts2, thickness=0.05, overlap=0.9, arc_samples=300, device=device)

# 4) sample curve
means1, thicknesses1 = bez1()
means2, thicknesses2 = bez2()
viewmat = w2c.to(dtype=means1.dtype)
K = K.to(dtype=means1.dtype)
# 5) render all spheres along the curve
means = torch.cat([means1, means2], dim=0)  # (N,3)
thicknesses = torch.cat([thicknesses1, thicknesses2], dim=0)  # (N,)
means = means.to(device)
thicknesses = thicknesses.to(device)

means = repeat(means, 'n c -> b n c', b=batch_size)  # (B,N,3)
thicknesses = repeat(thicknesses, 'n -> b n', b=batch_size)  # (B,N)

print("means.shape", means.shape)
print("thicknesses.shape", thicknesses.shape)



means.shape torch.Size([16, 327, 3])
thicknesses.shape torch.Size([16, 327])


In [19]:
print(K.shape)

torch.Size([3, 3])


In [32]:
img_t = rasterize_spheres(
    means=means,
    radii_world=thicknesses,
    viewmat=viewmat,
    K=K,
    width=W,
    height=H
)  # (H,W,1)

print("img_t.shape", img_t.shape)

# 6) convert & show
img_np = (img_t[...,0].cpu().detach().numpy()*255).astype("uint8")
for i in range(batch_size):
    # show image inline in notebook
    plt.imshow(img_np[i], cmap='gray')
    plt.axis('off')
    plt.show()

OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacity of 11.74 GiB of which 465.62 MiB is free. Including non-PyTorch memory, this process has 9.57 GiB memory in use. Of the allocated memory 9.36 GiB is allocated by PyTorch, and 49.29 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)